-csv datei einlesen
-Name ändern
- 1sten 5 zeilen ausgeben

In [1]:
import pandas as pd
from datetime import datetime, timedelta
csv_file = '../Data_Cleanup/outCSV/Clean_Metro_Set.csv'
metro = pd.read_csv(csv_file)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/_v/6rn4knv13fs88z1_74ms8bk40000gn/T/ipykernel_44164/554973988.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metro = pd.read_csv(csv_file)


#### Sort Table by Transaction Time
Now we are going

In [2]:
metro['start_time'] = pd.to_datetime(metro['start_time'])

metro ['end_time'] = pd.to_datetime(metro['end_time'])

metro = metro.sort_values(by='start_time')

-Neue Spalte hinzufügen für Idle_Time
-Alle neuen felder mit "None" initialisieren, da alle nicht belegte Idle Time Zellen keinen Einfluss später in der durchschnittlichen Idle Time Berechnung haben sollten.


In [3]:
metro['idle_time'] = None

Idle Time berechnung

Erklärung des folgenden Codes:

Die Berechnung der Idle Time basiert auf der Überprüfung der Startzeit (current_start_time) im Vergleich zur Endzeit (stored end time) des vorherigen Vorgangs mit der gleichen Startstation (current_start_station_id).

Der Algorithmus durchläuft die Datenzeilen des DataFrames und speichert die Endzeiten (current_end_time) für jede Endstation (current_end_station_id) in einem Dictionary namens BigDict.

Für jede Zeile wird überprüft, ob die aktuelle Startzeit größer ist als die gespeicherte Endzeit für die Startstation. Wenn dies der Fall ist, bedeutet dies, dass zwischen den Vorgängen mit der gleichen Startstation eine Leerlaufzeit aufgetreten ist. Die Leerlaufzeit wird berechnet, indem die Startzeit von der gespeicherten Endzeit subtrahiert wird. Das Ergebnis wird dann in der Spalte "idle_time" der entsprechenden Zeile des DataFrames gespeichert.

Um die Endzeit für eine Startstation abzurufen, wird das Dictionary BigDict verwendet. Wenn eine Startstation zum ersten Mal auftritt und keine gespeicherte Endzeit vorhanden ist, wird ein KeyError ausgelöst. In diesem Fall wird die aktuelle Endzeit für die aktuelle Endstation dem BigDict hinzugefügt.

Durch dieses Verfahren wird für jede Zeile im DataFrame die Leerlaufzeit berechnet, sofern eine vorherige Operation mit derselben Startstation vorhanden ist. Dadurch können Informationen über Leerlaufzeiten zwischen aufeinanderfolgenden Vorgängen mit derselben Startstation erfasst und im DataFrame gespeichert werden.

In [4]:
BigDict = {
    # station_id: {
    #     bike_id: (transaction_id ,end_time)
    # }
}

for i, row in metro.iterrows():
    current_start_time = row["start_time"]
    current_end_time = row["end_time"]
    current_end_station_id = row["end_station_id"]
    current_start_station_id = row["start_station_id"]
    current_bike_id = row["bike_id"]
    current_index = i

    if current_start_station_id in BigDict:
        for bike in list(BigDict[current_start_station_id]):  # Create a copy of the keys
            idle_time = current_start_time - BigDict[current_start_station_id][bike][1]
            metro.at[BigDict[current_start_station_id][bike][0], "idle_time"] = idle_time
            if idle_time.total_seconds() >= 0:
                del BigDict[current_start_station_id][bike]

    if current_end_station_id not in BigDict:
        BigDict[current_end_station_id] = {current_bike_id: (current_index, current_end_time)}

    BigDict[current_end_station_id][current_bike_id] = (current_index, current_end_time)

In [5]:
metro.to_csv('../Data_Cleanup/outCSV/Metro_Set_with_IdleTime.csv')